# Use IMAGELIST as camera source

## Import some libraries

In [1]:
import donkeycar as dk
import os

________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v5.0.dev3 ...


In [2]:
%run functions_donkey_car_parts.ipynb

## Loading myconfig

In [3]:
cfg = dk.load_config(config_path="/home/pi/mycar/config.py")

INFO:donkeycar.config:loading config file: /home/pi/mycar/config.py
INFO:donkeycar.config:loading personal config over-rides from myconfig.py


In [4]:
V = dk.vehicle.Vehicle()

In [ ]:
cfg.DRIVE_LOOP_HZ = 20

## Define Camera Helper function

In [5]:
data_path = "/home/pi/donkeycar-notebook/basic/resource"
tub_path = os.path.join(data_path, 'red_line_tub')
images_path = os.path.join(tub_path, 'images')

print(images_path)

cfg.PATH_MASK = images_path + "/*.jpg"

/home/pi/donkeycar-notebook/basic/resource/red_line_tub/images


In [6]:
# Override the get_camera function to use a ImageListCamera

def get_camera(cfg):
    from donkeycar.parts.camera import ImageListCamera
    cam = ImageListCamera(path_mask=cfg.PATH_MASK)

    return cam

In [7]:
add_camera(V, cfg)

INFO:donkeycar.parts.camera:2136 images loaded.
INFO:donkeycar.parts.camera:['/home/pi/donkeycar-notebook/basic/resource/red_line_tub/images/0_cam_image_array_.jpg', '/home/pi/donkeycar-notebook/basic/resource/red_line_tub/images/1_cam_image_array_.jpg', '/home/pi/donkeycar-notebook/basic/resource/red_line_tub/images/2_cam_image_array_.jpg', '/home/pi/donkeycar-notebook/basic/resource/red_line_tub/images/3_cam_image_array_.jpg', '/home/pi/donkeycar-notebook/basic/resource/red_line_tub/images/4_cam_image_array_.jpg', '/home/pi/donkeycar-notebook/basic/resource/red_line_tub/images/5_cam_image_array_.jpg', '/home/pi/donkeycar-notebook/basic/resource/red_line_tub/images/6_cam_image_array_.jpg', '/home/pi/donkeycar-notebook/basic/resource/red_line_tub/images/7_cam_image_array_.jpg', '/home/pi/donkeycar-notebook/basic/resource/red_line_tub/images/8_cam_image_array_.jpg', '/home/pi/donkeycar-notebook/basic/resource/red_line_tub/images/9_cam_image_array_.jpg']
INFO:donkeycar.vehicle:Adding par

## Define web controller helper function

In [8]:
add_user_controller(V, cfg, None, 'cam/image_array')

INFO:donkeycar.parts.web_controller.web:Starting Donkey Server...
INFO:donkeycar.parts.web_controller.web:You can now go to donkey-52a684.local:8887 to drive your car.
INFO:donkeycar.vehicle:Adding part LocalWebController.


In [9]:
## Putting them together

# Get wireless LAN IP address
import netifaces as ni
from IPython.display import HTML
ip = ni.ifaddresses('wlan0')[ni.AF_INET][0]['addr']
url = f"http://{ip}:{cfg.WEB_CONTROL_PORT}"
display(HTML(f'You can now go to <a href="{url}" target="_blank">{url}</a> to drive your car.'))

V.start(max_loop_count=100)

INFO:donkeycar.vehicle:Starting vehicle at 10 Hz
INFO:tornado.access:304 GET /drive (192.168.8.213) 7.44ms
INFO:tornado.access:304 GET /drive (192.168.8.213) 7.44ms
INFO:tornado.access:101 GET /wsDrive (192.168.8.213) 1.49ms
INFO:tornado.access:101 GET /wsDrive (192.168.8.213) 1.49ms
INFO:donkeycar.parts.web_controller.web:New client connected
INFO:donkeycar.parts.web_controller.web:Client disconnected
INFO:donkeycar.vehicle:Vehicle executed 700 steps in 70.05688381195068 seconds.
INFO:donkeycar.vehicle:Shutting down vehicle and its parts...
INFO:donkeycar.vehicle:Part Profile Summary: (times in ms)
INFO:donkeycar.vehicle:
+--------------------+------+------+------+------+------+------+-------+
|        part        | max  | min  | avg  | 50%  | 90%  | 99%  | 99.9% |
+--------------------+------+------+------+------+------+------+-------+
|  ImageListCamera   | 9.50 | 0.90 | 1.58 | 1.53 | 2.77 | 3.29 |  5.52 |
| LocalWebController | 1.89 | 0.13 | 0.25 | 0.20 | 0.36 | 1.55 |  1.87 |
+---

(700, 70.05688381195068)

In [10]:
V.stop()

INFO:donkeycar.vehicle:Shutting down vehicle and its parts...
INFO:donkeycar.vehicle:Part Profile Summary: (times in ms)
INFO:donkeycar.vehicle:
+--------------------+------+------+------+------+------+------+-------+
|        part        | max  | min  | avg  | 50%  | 90%  | 99%  | 99.9% |
+--------------------+------+------+------+------+------+------+-------+
|  ImageListCamera   | 9.50 | 0.90 | 1.58 | 1.53 | 2.77 | 3.29 |  5.52 |
| LocalWebController | 1.89 | 0.13 | 0.25 | 0.20 | 0.36 | 1.55 |  1.87 |
+--------------------+------+------+------+------+------+------+-------+


In [11]:
!netstat -an -A inet | grep 8887

tcp        0      0 0.0.0.0:8887            0.0.0.0:*               LISTEN     


## Kill process listening to 8887

In [ ]:
!kill -9 $(lsof -t -i:8887)